!pip install scipy --quiet
!pip install tenacity --quiet
#!pip install tiktoken==0.3.3 --quiet
!pip install termcolor --quiet
!pip install openai --quiet
!pip install arxiv --quiet
!pip install pandas --quiet
!pip install PyPDF2 --quiet
!pip install tqdm --quiet
!pip install fitz --quiet

In [1]:
import os
import json
import regex as re  # type: ignore
import numpy as np  # type: ignore
from IPython.display import display, Markdown   # type: ignore

import faiss    # type: ignore
import fitz     # type: ignore

from openai import OpenAI       # type: ignore
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer           # type: ignore
from sentence_transformers import SentenceTransformer                   # type: ignore
from tenacity import retry, wait_random_exponential, stop_after_attempt # type: ignore

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
GPT_MODEL = "gpt-4-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"
client = OpenAI()

In [2]:
def extract_text_from_pdf(filepath):
    """Extracts text from each page of a PDF file and returns a list of (text, page_number) tuples."""
    texts = []
    with fitz.open(filepath) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf[page_num]
            texts.append((page.get_text("text"), page_num + 1))  # Store text with page number
    return texts

def create_faiss_index_from_pdfs(directory_path):
    """Creates a FAISS index from PDF files in a specified directory."""
    index = faiss.IndexFlatL2(384)  # Assuming a 384-dimension embedding model

    metadata = []  # Store metadata with document name and page number
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            filepath = os.path.join(directory_path, filename)
            texts = extract_text_from_pdf(filepath)
            for text, page in texts:
                embedding = model.encode(text).reshape(1, -1)
                index.add(embedding)  # Add embedding to FAISS index
                metadata.append({"document": filename, "page": page})

    # Save index and metadata
    faiss.write_index(index, 'faiss_index.bin')
    np.save('metadata.npy', metadata)
    print("FAISS index and metadata saved.")

create_faiss_index_from_pdfs("C:/Users/Sasi/Downloads/Learnr/Knowledge_Base/PDFs")

# Load the retrieval and generation model
retrieval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
generation_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Load FAISS index and metadata
index = faiss.read_index('faiss_index.bin')
metadata = np.load('metadata.npy', allow_pickle=True)


FAISS index and metadata saved.


## Configure Agent

We'll create our agent in this step, including a ```Conversation``` class to support multiple turns with the API, and some Python functions to enable interaction between the ```ChatCompletion``` API and our knowledge base functions.

In [3]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            functions=functions,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
from termcolor import colored # type: ignore

class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                    (
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [5]:
analog_design_functions = [
    {
        "name": "calculate_pole_frequency",
        "description": """Calculate the pole (cutoff) frequency of a circuit based on resistance and capacitance values. This function is commonly used 
                        to determine the cutoff point for filters.""",
        "parameters": {
            "type": "object",
            "properties": {
                "resistance": {
                    "type": "number",
                    "description": "The resistance in ohms (Ω) for the circuit."
                },
                "capacitance": {
                    "type": "number",
                    "description": "The capacitance in farads (F) for the circuit."
                }
            },
            "required": ["resistance", "capacitance"],
        },
    },
    {
        "name": "two_port_amplifier_gain",
        "description": """Calculate the output voltage and gain of a two-port amplifier model. This is useful for analyzing amplifier circuits 
                        and determining the output for given input and circuit parameters.""",
        "parameters": {
            "type": "object",
            "properties": {
                "Rin": {
                    "type": "number",
                    "description": "Input resistance in ohms (Ω)."
                },
                "A": {
                    "type": "number",
                    "description": "Amplifier gain, typically dimensionless."
                },
                "Rout": {
                    "type": "number",
                    "description": "Output resistance in ohms (Ω)."
                },
                "Vin": {
                    "type": "number",
                    "description": "Input voltage in volts (V)."
                }
            },
            "required": ["Rin", "A", "Rout", "Vin"],
        },
    },
    {
        "name": "sctc_octc_analysis",
        "description": """Determine if a capacitor should be treated as an open or short circuit for SCTC (Short-Circuit Time Constant) or OCTC (Open-Circuit Time Constant) analysis 
                        based on its type in the circuit.""",
        "parameters": {
            "type": "object",
            "properties": {
                "capacitor_type": {
                    "type": "string",
                    "description": "Type of capacitor usage, specify 'high-pass' or 'low-pass'."
                }
            },
            "required": ["capacitor_type"],
        },
    },
    {
        "name": "rag_retrieval_and_generation",
        "description": """This function uses Retrieval-Augmented Generation (RAG) to answer user queries about analog electronics. 
                          It retrieves relevant information from a FAISS index of document embeddings, constructs a response based on the context, 
                          and cites the source document and page number for reference.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": """
                        A user question regarding analog electronics. The function will search the FAISS index for relevant information, 
                        generate an answer based on the retrieved context, and cite the source document and page number.
                    """,
                },
                "top_k": {
                    "type": "integer",
                    "description": """
                        The number of top relevant passages to retrieve from the FAISS index. Increasing this number may provide more context but could 
                        affect performance and relevance.
                    """,
                    "default": 3
                },
            },
            "required": ["query"]
        }
    }
]


In [6]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.choices[0]
    if full_message.finish_reason == "function_call":
        print(f"Function generation requested, calling function")
        return call_analog_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response

def call_analog_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    function_name = full_message.message.function_call.name
    try:
        # Parse the arguments from JSON format
        parsed_output = json.loads(full_message.message.function_call.arguments)
        print(full_message.message.function_call)
    except json.JSONDecodeError as e:
        print("Failed to parse function arguments.")
        print(f"Error message: {e}")
        raise

    results = None  # Initialize results

    # Handle different functions based on the function name
    if function_name == "calculate_pole_frequency":
        try:
            print("Calculating pole frequency")
            results = calculate_pole_frequency(
                parsed_output["resistance"],
                parsed_output["capacitance"]
            )
        except Exception as e:
            print(f"Function execution failed for calculate_pole_frequency.")
            print(f"Error message: {e}")

    elif function_name == "two_port_amplifier_gain":
        try:
            print("Calculating two-port amplifier gain")
            results = two_port_amplifier_gain(
                parsed_output["Rin"],
                parsed_output["A"],
                parsed_output["Rout"],
                parsed_output["Vin"]
            )
        except Exception as e:
            print(f"Function execution failed for two_port_amplifier_gain.")
            print(f"Error message: {e}")

    elif function_name == "sctc_octc_analysis":
        try:
            print("Performing SCTC/OCTC analysis")
            results = sctc_octc_analysis(parsed_output["capacitor_type"])
        except Exception as e:
            print(f"Function execution failed for sctc_octc_analysis.")
            print(f"Error message: {e}")

    elif function_name == "rag_retrieval_and_generation":
        try:
            print("Learning from course documents")
            results = calculate_pole_frequency(parsed_output["query"])
        except Exception as e:
            print(f"Function execution failed for rag_retrieval_and_generation.")
            print(f"Error message: {e}")
            
    else:
        raise Exception(f"Function {function_name} does not exist and cannot be called")

    # Append the results as a message in the conversation
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": str(results),
        }
    )

    # Generate the response using the results
    try:
        print(f"Got results for {function_name}, generating response")
        response = chat_completion_request(messages)
        return response
    except Exception as e:
        print("Function chat request failed.")
        print(f"Error message: {e}")
        raise

def calculate_pole_frequency(resistance, capacitance):
    """
    Calculate the pole frequency (cutoff frequency) for a low-pass or high-pass filter circuit.
    
    Parameters:
    - resistance (float): The resistance value in ohms (Ω).
    - capacitance (float): The capacitance value in farads (F).
    
    Returns:
    - float: The pole frequency (ω_p) in radians per second (rad/s).
    """
    tau = resistance * capacitance  # Time constant (τ)
    omega_p = 1 / tau  # Pole frequency
    return omega_p

def two_port_amplifier_gain(Rin, A, Rout, Vin):
    """
    Calculate the output voltage and gain of a two-port amplifier circuit model.
    
    Parameters:
    - Rin (float): Input resistance in ohms (Ω).
    - A (float): Amplifier gain (dimensionless).
    - Rout (float): Output resistance in ohms (Ω).
    - Vin (float): Input voltage in volts (V).
    
    Returns:
    - tuple (float, float): Output voltage (Vout) and gain (A).
    """
    Vout = A * Vin * (Rin / (Rin + Rout))
    return Vout, A

def sctc_octc_analysis(capacitor_type, capacitances, resistances):
    """
    Perform Short Circuit Time Constant (SCTC) and Open Circuit Time Constant (OCTC) analysis.
    
    Parameters:
    - capacitor_type (str): Either 'high-pass' or 'low-pass'.
    - capacitances (list of float): Capacitance values in farads for each capacitor in the circuit.
    - resistances (list of float): Corresponding resistance values in ohms seen by each capacitor.
    
    Returns:
    - dict: A dictionary with estimated higher or lower cutoff frequency based on SCTC/OCTC analysis.
    """
    # Check if the input lists for capacitances and resistances match
    if len(capacitances) != len(resistances):
        raise ValueError("Capacitances and resistances lists must have the same length.")
    
    # Perform calculation based on the capacitor type
    if capacitor_type == "high-pass":
        # SCTC: Use the formula for lower cutoff frequency (ωL-3dB)
        lower_cutoff_freq = sum(1 / (cap * res) for cap, res in zip(capacitances, resistances))
        return {
            "cutoff_frequency": lower_cutoff_freq,
            "type": "SCTC (Short Circuit Time Constant)",
            "description": "Estimated lower cutoff frequency for high-pass configuration."
        }
    
    elif capacitor_type == "low-pass":
        # OCTC: Use the formula for higher cutoff frequency (ωH-3dB)
        higher_cutoff_freq = 1 / sum(cap * res for cap, res in zip(capacitances, resistances))
        return {
            "cutoff_frequency": higher_cutoff_freq,
            "type": "OCTC (Open Circuit Time Constant)",
            "description": "Estimated higher cutoff frequency for low-pass configuration."
        }
    
    else:
        return {
            "error": "Invalid capacitor type. Use 'high-pass' or 'low-pass'."
        }
    
def rag_retrieval_and_generation(query, top_k=3):
    """Retrieve relevant passages and generate an answer with citations."""
    # Encode query and retrieve nearest neighbors
    query_embedding = retrieval_model.encode(query).reshape(1, -1)
    _, indices = index.search(query_embedding, top_k)  # Get indices of top-k matches
    
    # Retrieve relevant texts and their metadata
    retrieved_texts = []
    citations = []
    for idx in indices[0]:
        doc_info = metadata[idx]
        document_name = doc_info["document"]
        page_number = doc_info["page"]
        citation = f"{document_name}, page {page_number}"
        retrieved_texts.append(doc_info["text"])
        citations.append(citation)
    
    # Concatenate retrieved texts for the generative model
    context = " ".join(retrieved_texts)
    
    # Generate answer using a sequence-to-sequence model
    inputs = tokenizer(f"Answer based on context: {context} Query: {query}", return_tensors="pt", truncation=True)
    output = generation_model.generate(**inputs, max_length=150, num_beams=3, early_stopping=True)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Format answer with citations
    formatted_answer = f"{answer}\n\nSources:\n" + "\n".join(citations)
    return formatted_answer


In [ ]:
# Start with a system message
paper_system_message = "You are a useful analog electronics tutor. Use the content and functions provided to guide students to ideate, think critically and solve analog electronincs problems in a fast and effective way?"
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)

# Add a user message
paper_conversation.add_message("user", "Calculate the pole frequency for a circuit with a resistance of 1000 ohms and a capacitance of 0.000001 farads.")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=analog_design_functions
)

assistant_message = chat_response.choices[0].message.content
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Function not required, responding to user


To calculate the pole frequency (cutoff frequency) of a circuit comprising a resistor and a capacitor, you would use the formula:

\[
f_c = \frac{1}{2\pi RC}
\]

Where:
- \( R \) is the resistance in ohms (\(\Omega\)).
- \( C \) is the capacitance in farads (F).
- \( f_c \) is the cutoff frequency in hertz (Hz).

Given:
- \( R = 1000 \) ohms
- \( C = 0.000001 \) farads

Can you apply the formula to calculate the cutoff frequency?

In [8]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    """Please perform an SCTC/OCTC analysis for a low-pass filter with the following capacitor-resistor pairs:

    Capacitances: 1e-6 F, 2e-6 F, 0.5e-6 F
    Resistances: 1000 Ω, 2000 Ω, 1500 Ω."""
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=analog_design_functions
)
display(Markdown(updated_response.choices[0].message.content))


Function not required, responding to user


To determine whether each capacitor in the low-pass filter configuration should be treated as an open or short circuit for SCTC (Short-Circuit Time Constant) and OCTC (Open-Circuit Time Constant) analysis, we'll consider the behavior of the capacitor for each capacitor-resistor pair you provided:

1. Capacitance: \(1 \times 10^{-6}\) F, Resistance: 1000 Ω
2. Capacitance: \(2 \times 10^{-6}\) F, Resistance: 2000 Ω
3. Capacitance: \(0.5 \times 10^{-6}\) F, Resistance: 1500 Ω

For a low-pass filter whic uses the capacitor:

Think about how capacitors behave in low-frequency and high-frequency scenarios in this filtering context. Think about how you would consider the speed of voltage change across the capacitor at different frequencies and how that affects your analysis of treating it as open or closed. 

What do you think should be the treatment of each capacitor in the SCTC and OCTC analyses?

In [ ]:
# Start with a system message
chatbot_name = "AnalogBot"
dt = "Monday 10th October 2024"
course_code = "EE2102"
course_title = "Analog Electronics"
topics_list = "SOLID-STATE DIODES AND DIODE CIRCUITS,SMALL-SIGNAL MODELING AND LINEAR AMPLIFICATION,NONIDEAL OPERATIONAL AMPLIFIERS AND FEEDBACK AMPLIFIER STABILITY,FIELD-EFFECT TRANSISTORS,OPERATIONAL AMPLIFIER APPLICATIONS,SINGLE-TRANSISTOR AMPLIFIERS,DIFFERENTIAL AMPLIFIERS AND OPERATIONAL AMPLIFIER DESIGN,TRANSISTOR FEEDBACK AMPLIFIERS AND OSCILLATORS,INTRODUCTION TO DIGITAL ELECTRONICS,BIPOLAR LOGIC CIRCUITS,SOLID-STATE ELECTRONICS,INTRODUCTION TO ELECTRONICS,ANALOG INTEGRATED CIRCUIT DESIGN TECHNIQUES,MOS MEMORY AND STORAGE CIRCUITS,COMPLEMENTARY MOS (CMOS) LOGIC DESIGN,BIPOLAR JUNCTION TRANSISTORS,AMPLIFIER FREQUENCY RESPONSE,ANALOG SYSTEMS AND IDEAL OPERATIONAL AMPLIFIERS"

paper_system_message = f"""  ## Your role - Today is {dt}. 
- You are {chatbot_name}, a tutor for {course_title} course, with course code {course_code}, at Nanyang Technological University. The topics covered in the course are {topics_list}.   
- Your task is to only answer questions that are related to {course_title}, and the associated topics as listed in {topics_list}. For other types of questions, you **must respond**, "Apologies. I do not have the answer to this question. Please try another query or topic." 
- Be clear in every step, idea and calculation provided
- Utilize the FAISS embedding to retrieve the most relevant information, and cite your sources to the best of your ability when using uploaded reference materials.
- You must use Python programming functions to provide accurate calculations for any problems associated to {course_title} as provided by the user.
- You must always hide the calculated solution from the user. Instead, make the user think in a Socratic manner to solve the questions with minimal assistance. You may provide formulae, ideas or theorems to encourage the user to find their own solution.
"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)

# Add a user message
paper_conversation.add_message("user", "Calculate the pole frequency for a circuit with a resistance of 1000 ohms and a capacitance of 0.000001 farads.")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=analog_design_functions
)

assistant_message = chat_response.choices[0].message.content
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Function not required, responding to user


To calculate the pole frequency (cutoff frequency) of a circuit comprising a resistor and a capacitor, you would use the formula:

\[
f_c = \frac{1}{2\pi RC}
\]

Where:
- \( R \) is the resistance in ohms (\(\Omega\)).
- \( C \) is the capacitance in farads (F).
- \( f_c \) is the cutoff frequency in hertz (Hz).

Given:
- \( R = 1000 \) ohms
- \( C = 0.000001 \) farads

Can you apply the formula to calculate the cutoff frequency?

In [ ]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    """Please perform an SCTC/OCTC analysis for a low-pass filter with the following capacitor-resistor pairs:

    Capacitances: 1e-6 F, 2e-6 F, 0.5e-6 F
    Resistances: 1000 Ω, 2000 Ω, 1500 Ω."""
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=analog_design_functions
)
display(Markdown(updated_response.choices[0].message.content))


Function not required, responding to user


To determine whether each capacitor in the low-pass filter configuration should be treated as an open or short circuit for SCTC (Short-Circuit Time Constant) and OCTC (Open-Circuit Time Constant) analysis, we'll consider the behavior of the capacitor for each capacitor-resistor pair you provided:

1. Capacitance: \(1 \times 10^{-6}\) F, Resistance: 1000 Ω
2. Capacitance: \(2 \times 10^{-6}\) F, Resistance: 2000 Ω
3. Capacitance: \(0.5 \times 10^{-6}\) F, Resistance: 1500 Ω

For a low-pass filter whic uses the capacitor:

Think about how capacitors behave in low-frequency and high-frequency scenarios in this filtering context. Think about how you would consider the speed of voltage change across the capacitor at different frequencies and how that affects your analysis of treating it as open or closed. 

What do you think should be the treatment of each capacitor in the SCTC and OCTC analyses?